# Ensemble learning

Ensemble learning is een manier om meerdere machine learning algorithmes te gaan combineren om een beter resultaat te bekomen.
Een voorbeeld hiervan is Random Forest dat een ensemble is van een aantal decision trees. 
Een belangrijke opmerking hierbij is dat meerdere soorten machine learning algoritmes gecombineerd kunnen worden.
Er zijn hier drie bekende varianten van, namelijk:
* Stacking
* Bagging
* Boosting

Deze varianten gaan we nu 1 voor 1 behandelen.

# Stacking 

Een schets van hoe een stacking-ensemble werkt zie je hieronder.
Bij de stacking-methode wordt de volledige trainingsdata gebruikt om meerdere modellen te trainen.
Deze kunnen maar moeten niet van hetzelfde algoritme zijn en ook de hyperparameters kunnen verschillen.
Nadat al deze modellen getrained zijn, wordt er nog model getrained dat een selectie maakt van wat de uiteindelijke voorspelling is op basis van de uitkomst van alle getrainde modellen.
Dit tweede model kan dus evalueren wanneer welk model het meest correct is.
Het algoritme hiervoor kan vrij gekozen worden en kan zelfs een ander ensemble zijn.
Bij stacking is dus niet strikt majority voting van toepassing.

![ensemble](images\stacking.png)

Een [stacking ensemble](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html) kan als volgt geimplementeerd worden in sklearn:

In [5]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

estimators = [
    ("rf", RandomForestClassifier(n_estimators=1000)),
    ("lr", LogisticRegression(C=100)),
    ("knn", KNeighborsClassifier()),
    ("svm", SVC(C=10))
]

stack = StackingClassifier(estimators, LogisticRegression(), passthrough=True)  # passthrough (data ook naar de final estimator)
%time stack.fit(X_train, y_train)
%time stack.score(X_test,y_test)

CPU times: total: 3.73 s
Wall time: 3.76 s
CPU times: total: 125 ms
Wall time: 134 ms


0.9666666666666667

# Bagging

De tweede methode bagging is ook gekend onder de naam bootstrap bagging.
Dit houdt in dat elk model slechts getrained wordt op een deel van de data (bootstrapped samples).
Deze observaties om elk model te trainen worden willekeurig gekozen met teruglegging.
Typisch wordt er ongeveer 60% van de totale data gebruikt om elk model te trainen.
Wanneer elk model getrained is wordt er een standaard majority voting toegepast om de uiteindelijke predictie te bekomen (bij regressie wordt er het gemiddelde genomen).

![bagging](images\bagging.png)

Een belangrijke opmerking is dat theoretisch gezien meerdere types van machine learning algoritmes gecombineerd kunnen worden maar dit dit niet rechtstreeks gaat met sklearn zonder zelf een ensemble te implementeren. 
Dit is ook niet nodig omdat zolang je ensemble groot genoeg is, je elke gewenste accuraatheid kan bereiken.
Een implementatie van de [bagging methode](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html) ziet er als volgt uit:

In [11]:
# create bagging classifier
from sklearn.ensemble import BaggingClassifier

bagging = BaggingClassifier(SVC(), n_estimators=200, max_samples=0.6, max_features=0.4)

%time bagging.fit(X_train, y_train)
%time bagging.score(X_test, y_test)

import time
start = time.time()
bagging.fit(X_train, y_train)
end = time.time()
print(end-start)

CPU times: total: 172 ms
Wall time: 171 ms
CPU times: total: 15.6 ms
Wall time: 25 ms
0.16488289833068848


# Boosting

De derde variant van ensemble learning methoden is boosting.
Dit is een aanpassing van de bagging methode waar de bootstrapped samples niet meer willekeurig zijn.
Bij boosting worden de modellen sequentieel getrained en gevalideerd (met de trainingsdata diet niet in de bootstrapped sample zit). 
De classificaties die verkeerd waren bij deze validatie stap hebben een grotere kans om in de bootstrapped sample te zitten van het volgende model.
Omdat de uitkomst van elk model nodig is voor het volgende kan dit niet geparallelliseerd worden waardoor de trainingstijd snel kan oplopen.
Het voordeel echter van deze methode is dat de accuraatheid van het gecombineerde model hoger gaat zijn dan bij bagging.
De meest bekende implementatie van deze methode wordt AdaBoost genoemd.
Daarnaast wordt tegenwoordig ook XGBoost benoemd wat staat voor Extreme Gradient Boosting.
Het probleem is echter dat deze techniek niet standaard in sklearn staat.
Hiervoor moet een extra package toegevoegd worden, meer informatie hierover vind je [hier](https://towardsdatascience.com/getting-started-with-xgboost-in-scikit-learn-f69f5f470a97).
Deze techniek is een speciale variant van een random forest (er wordt gewerkt met meerdere decision trees).
Meer informatie over de api vind je [hier](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier).
De voordelen van xgboost zijn:
* Hoge accuraatheid
* Heel snelle uitvoering door parallellisatie
* Flexibel algoritme door keuse van optimalisatie
* Kan omgaan met missing data
* Voert autmatisch pruning uit om overfitting tegen te gaan
* Ingebouwde cross-validatie

![boosting](images\boosting.png)

 Een implementatie van de [boosting methode](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) ziet er als volgt uit:

In [17]:
# create bagging classifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

boosting = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators=2000000, algorithm='SAMME')

%time boosting.fit(X_train, y_train)
%time boosting.score(X_test, y_test)

import time
start = time.time()
boosting.fit(X_train, y_train)
end = time.time()
print(end-start)
boosting.score(X_test, y_test)

CPU times: total: 15.6 ms
Wall time: 8.98 ms
CPU times: total: 15.6 ms
Wall time: 10 ms
0.008998632431030273


0.9333333333333333

Meer informatie over alle mogelijke ensemble-methoden in sklearn vind je [hier](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble).

# XGBoost

Gradient Boosting is een ML-techniek die iteratief modellen toevoegt aan een ensemble.
Er wordt begonnen met een eenvoudig model dat zeer naief is.
De voorspellingen van dit model worden gebruikt om een kost-functie te berekenen.
Deze functie wordt dan gebruikt voor een nieuw model te trainen dat toegevoegd aan het ensemble.
De parameters van dit nieuwe model worden zo gekozen dat de kost-functie verminderd.
Met dit uitgebreide ensemble worden opnieuw voorspelling gedaan die het dan terug gebruikt worden om een kostfunctie te berekenen.

De belangrijkste parameters van deze techniek zijn:
* n_estimators: Geeft het maximum aantal bomen in het ensemble weer. Een lage waarde kan leiden tot underfitting, een grote tot overfitting.
* learning rate: Dit kan gebruikt zorden om de impact van extra bomen te verkleinen om overfitting tegen te gaan. Lagere waarden gaan normaal een hogere accuraatheid maar ook een hogere trainingstijd opleveren.
* early stopping: Deze parameter moet meegegeven worden bij het fitten en kan gebruikt worden om overfitting tegen te gaan. Indien de validation score niet afneemt gedurende dit aantal rondes, dan stopt de fit-functie omdat er overfitting gedetecteerd wordt.
* eval_set: Deze parameter is een tuple van de validatieset dat gebruikt kan worden voor het early-stopping te testen. Deze moet dus met early-stopping gecombineerd worden.
* n_jobs: aantal cores dat kan gebruikt worden voor de training


Meer informatie over de parameters van deze techniek vind je [hier](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn).

In [20]:
# create xgboost classifier
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators = 1000, learning_rate=0.05, early_stopping_rounds=5)

%time xgb.fit(X_train, y_train, eval_set=[(X_test,y_test)])     # groot verschil tussen CPU times en wall time -> parallel algoritme
xgb.score(X_test, y_test)

[0]	validation_0-mlogloss:1.03254
[1]	validation_0-mlogloss:0.97194
[2]	validation_0-mlogloss:0.91617
[3]	validation_0-mlogloss:0.86469
[4]	validation_0-mlogloss:0.81704
[5]	validation_0-mlogloss:0.77284
[6]	validation_0-mlogloss:0.73175
[7]	validation_0-mlogloss:0.69437
[8]	validation_0-mlogloss:0.65869
[9]	validation_0-mlogloss:0.62608
[10]	validation_0-mlogloss:0.59565
[11]	validation_0-mlogloss:0.56786
[12]	validation_0-mlogloss:0.54123
[13]	validation_0-mlogloss:0.51617
[14]	validation_0-mlogloss:0.49337
[15]	validation_0-mlogloss:0.47126
[16]	validation_0-mlogloss:0.45060
[17]	validation_0-mlogloss:0.43106
[18]	validation_0-mlogloss:0.41334
[19]	validation_0-mlogloss:0.39601
[20]	validation_0-mlogloss:0.37898
[21]	validation_0-mlogloss:0.36343
[22]	validation_0-mlogloss:0.34837
[23]	validation_0-mlogloss:0.33509
[24]	validation_0-mlogloss:0.32311
[25]	validation_0-mlogloss:0.31092
[26]	validation_0-mlogloss:0.30030
[27]	validation_0-mlogloss:0.28942
[28]	validation_0-mlogloss:0.2

0.9666666666666667